# 单个数据预处理

为构建我们的本地知识库，我们需要对以多种类型存储的本地文档进行处理，读取本地文档并通过前文描述的 Embedding 方法将本地文档的内容转化为词向量来构建向量数据库。接下来以一些实际示例入手，来讲解如何对本地文档进行处理。
## 数据读取
将知识库源数据放置在../../data_base/knowledge_db 目录下

**PDF文档为例**  
可以使用 LangChain 的 PyMuPDFLoader 来读取知识库的 PDF 文件。PyMuPDFLoader 是 PDF 解析器中速度最快的一种，结果会包含 PDF 及其页面的详细元数据，并且每页返回一个文档。

In [10]:
from langchain.document_loaders.pdf import PyMuPDFLoader

# 创建一个 PyMuPDFLoader Class 实例，输入为待加载的 pdf 文档路径
loader = PyMuPDFLoader("../../data_base/knowledge_path/pumkin_book/pumpkin_book.pdf")

# 调用 PyMuPDFLoader Class 的函数 load 对 pdf 文件进行加载
pdf_pages = loader.load()

文档加载后储存在 `pdf_pages` 变量中:
- `pdf_pages` 的变量类型为 `List`
- 打印 `pdf_pages` 的长度可以看到 pdf 一共包含多少页

In [11]:
print(f"载入后的变量类型为：{type(pdf_pages)}，",  f"该 PDF 一共包含 {len(pdf_pages)} 页")

载入后的变量类型为：<class 'list'>， 该 PDF 一共包含 196 页


`pdf_pages` 中的每一元素为一个文档，变量类型为 `langchain_core.documents.base.Document`, 文档变量类型包含两个属性
- `page_content` 包含该文档的内容。
- `meta_data` 为文档相关的描述性数据。

In [13]:
pdf_page = pdf_pages[1]
print(f"每一个元素的类型：{type(pdf_page)}.", 
    f"该文档的描述性数据：{pdf_page.metadata}", 
    f"查看该文档的内容:\n{pdf_page.page_content}", 
    sep="\n------\n")

每一个元素的类型：<class 'langchain_core.documents.base.Document'>.
------
该文档的描述性数据：{'source': '../../data_base/knowledge_path/pumkin_book/pumpkin_book.pdf', 'file_path': '../../data_base/knowledge_path/pumkin_book/pumpkin_book.pdf', 'page': 1, 'total_pages': 196, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'LaTeX with hyperref', 'producer': 'xdvipdfmx (20200315)', 'creationDate': "D:20230303170709-00'00'", 'modDate': '', 'trapped': ''}
------
查看该文档的内容:
前言
“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读
者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推
导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充
具体的推导细节。”
读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周
老师之所以省去这些推导细节的真实原因是，他本尊认为“理工科数学基础扎实点的大二下学生应该对西瓜书
中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习”。所以...... 本南瓜书只能算是我
等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的“理工科数学基础扎实点的大二
下学生”。
使用说明
• 南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书
为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书；
• 对于初学机器学习的小白，西瓜书第1 章和第2 章的公式强烈不建议深究，简单过一下即可，等你学得
有点飘的时

In [14]:
# 假设环境中缺少 NLTK 数据包中的 punkt 资源，这是 NLTK 库中用于分词的一个重要组件。解决这个问题的方法是按照报错信息中提到的步骤下载并安装 punkt 资源。
# import nltk
# nltk.download('punkt', quiet=True)

## 三、数据清洗
我们期望知识库的数据尽量是有序的、优质的、精简的，因此我们要删除低质量的、甚至影响理解的文本数据。  
可以看到上文中读取的pdf文件不仅将一句话按照原文的分行添加了换行符`\n`，也在原本两个符号中间插入了`\n`，我们可以使用正则表达式匹配并删除掉`\n`。

In [15]:
import re
pattern = re.compile(r'[^\u4e00-\u9fff](\n)[^\u4e00-\u9fff]', re.DOTALL)
pdf_page.page_content = re.sub(pattern, lambda match: match.group(0).replace('\n', ''), pdf_page.page_content)
print(pdf_page.page_content)

前言
“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读
者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推
导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充
具体的推导细节。”
读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周
老师之所以省去这些推导细节的真实原因是，他本尊认为“理工科数学基础扎实点的大二下学生应该对西瓜书
中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习”。所以...... 本南瓜书只能算是我
等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的“理工科数学基础扎实点的大二
下学生”。
使用说明
• 南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书
为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书；• 对于初学机器学习的小白，西瓜书第1 章和第2 章的公式强烈不建议深究，简单过一下即可，等你学得
有点飘的时候再回来啃都来得及；• 每个公式的解析和推导我们都力(zhi) 争(neng) 以本科数学基础的视角进行讲解，所以超纲的数学知识
我们通常都会以附录和参考文献的形式给出，感兴趣的同学可以继续沿着我们给的资料进行深入学习；• 若南瓜书里没有你想要查阅的公式，或者你发现南瓜书哪个地方有错误，请毫不犹豫地去我们GitHub 的
Issues（地址：https://github.com/datawhalechina/pumpkin-book/issues）进行反馈，在对应版块
提交你希望补充的公式编号或者勘误信息，我们通常会在24 小时以内给您回复，超过24 小时未回复的
话可以微信联系我们（微信号：at-Sm1les）；
配套视频教程：https://www.bilibili.com/video/BV1Mh411e7VU
在线阅读地址：https://datawhalechina.github.io/pumpkin-book（仅供第1 版）
最新版PDF 获取地址：https://github.com/datawhalechina/pumpkin-book/releases
编委会
主

进一步分析数据，我们发现数据中还有不少的`•`和空格，我们的简单实用replace方法即可。

In [16]:
pdf_page.page_content = pdf_page.page_content.replace('•', '')
pdf_page.page_content = pdf_page.page_content.replace(' ', '')
print(pdf_page.page_content)

前言
“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读
者通过西瓜书对机器学习有所了解,所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推
导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充
具体的推导细节。”
读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周
老师之所以省去这些推导细节的真实原因是，他本尊认为“理工科数学基础扎实点的大二下学生应该对西瓜书
中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习”。所以......本南瓜书只能算是我
等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的“理工科数学基础扎实点的大二
下学生”。
使用说明
南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书
为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书；对于初学机器学习的小白，西瓜书第1章和第2章的公式强烈不建议深究，简单过一下即可，等你学得
有点飘的时候再回来啃都来得及；每个公式的解析和推导我们都力(zhi)争(neng)以本科数学基础的视角进行讲解，所以超纲的数学知识
我们通常都会以附录和参考文献的形式给出，感兴趣的同学可以继续沿着我们给的资料进行深入学习；若南瓜书里没有你想要查阅的公式，或者你发现南瓜书哪个地方有错误，请毫不犹豫地去我们GitHub的
Issues（地址：https://github.com/datawhalechina/pumpkin-book/issues）进行反馈，在对应版块
提交你希望补充的公式编号或者勘误信息，我们通常会在24小时以内给您回复，超过24小时未回复的
话可以微信联系我们（微信号：at-Sm1les）；
配套视频教程：https://www.bilibili.com/video/BV1Mh411e7VU
在线阅读地址：https://datawhalechina.github.io/pumpkin-book（仅供第1版）
最新版PDF获取地址：https://github.com/datawhalechina/pumpkin-book/releases
编委会
主编：Sm1les、archwalker

上文中读取的md文件每一段中间隔了一个换行符，我们同样可以使用replace方法去除。

## 四、文档分割

由于单个文档的长度往往会超过模型支持的上下文，导致检索得到的知识太长超出模型的处理能力，因此，在构建向量知识库的过程中，我们往往需要对文档进行分割，将单个文档按长度或者按固定的规则分割成若干个 chunk，然后将每个 chunk 转化为词向量，存储到向量数据库中。

在检索时，我们会以 chunk 作为检索的元单位，也就是每一次检索到 k 个 chunk 作为模型可以参考来回答用户问题的知识，这个 k 是我们可以自由设定的。

Langchain 中文本分割器都根据 `chunk_size` (块大小)和 `chunk_overlap` (块与块之间的重叠大小)进行分割。

![image.png](../../figures/C3-3-example-splitter.png)

* chunk_size 指每个块包含的字符或 Token （如单词、句子等）的数量

* chunk_overlap 指两个块之间共享的字符数量，用于保持上下文的连贯性，避免分割丢失上下文信息

Langchain 提供多种文档分割方式，区别在怎么确定块与块之间的边界、块由哪些字符/token组成、以及如何测量块大小

- RecursiveCharacterTextSplitter(): 按字符串分割文本，递归地尝试按不同的分隔符进行分割文本。
- CharacterTextSplitter(): 按字符来分割文本。
- MarkdownHeaderTextSplitter(): 基于指定的标题来分割markdown 文件。
- TokenTextSplitter(): 按token来分割文本。
- SentenceTransformersTokenTextSplitter(): 按token来分割文本
- Language(): 用于 CPP、Python、Ruby、Markdown 等。
- NLTKTextSplitter(): 使用 NLTK（自然语言工具包）按句子分割文本。
- SpacyTextSplitter(): 使用 Spacy按句子的切割文本。

In [14]:
''' 
* RecursiveCharacterTextSplitter 递归字符文本分割
RecursiveCharacterTextSplitter 将按不同的字符递归地分割(按照这个优先级["\n\n", "\n", " ", ""])，
    这样就能尽量把所有和语义相关的内容尽可能长时间地保留在同一位置
RecursiveCharacterTextSplitter需要关注的是4个参数：

* separators - 分隔符字符串数组
* chunk_size - 每个文档的字符数量限制
* chunk_overlap - 两份文档重叠区域的长度
* length_function - 长度计算函数
'''
#导入文本分割器
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [15]:
# 知识库中单段文本长度
CHUNK_SIZE = 500

# 知识库中相邻文本重合长度
OVERLAP_SIZE = 50

In [16]:
# 使用递归字符文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)
text_splitter.split_text(pdf_page.page_content[0:1000])

['法律声明\n本资料著作权属中兴通讯股份有限公司所有。未经著作权人书面许可，任何单位或个人不得以任何方式\n摘录、复制或翻译。\n侵权必究。\xa0\n和\xa0是中兴通讯股份有限公司的注册商标。中兴通讯产品的名称和标志是中兴通讯的专有\n标志或注册商标。在本手册中提及的其他产品或公司的名称可能是其各自所有者的商标或商名。在未经\n中兴通讯或第三方商标或商名所有者事先书面同意的情况下，本手册不以任何方式授予阅读者任何使用\n本手册上出现的任何标记的许可或权利。\n本产品符合关于环境保护和人身安全方面的设计要求，产品的存放、使用和弃置应遵照产品手册、相关\n合同或相关国法律、法规的要求进行。\n如果本产品进行改进或技术变更，恕不另行专门通知。\n当出现产品改进或者技术变更时，您可以通过中兴通讯技术支持网站http://support.zte.com.cn查询有\n关信息。\xa0\n第三方嵌入式软件使用限制声明：\n如果与本产品配套交付了Oracle、Sybase/SAP、Veritas、Microsoft、VMware、Redhat这些第三方嵌入',
 '式软件，只允许作为本产品的组件，与本产品捆绑使用。当本产品被废弃时，这些第三方软件的授权许\n可同时作废，不可转移。这些嵌入式软件由ZTE给最终用户提供技术支持。\n修订历史\n资料版本\n发布日期\n更新说明\nR1.22022-6-20\n更新菜单路径和界面图\nR1.12021-05-12\n更新菜单路径和界面图\nR1.02020-12-30\n第一次发布\n资料编号∶SJ-20220623151803-001\n发布日期∶2022-06-20（R1.2）']

In [17]:
split_docs = text_splitter.split_documents(pdf_pages)
print(f"切分后的文件数量：{len(split_docs)}")

切分后的文件数量：21


In [18]:
print(f"切分后的字符数（可以用来大致评估 token 数）：{sum([len(doc.page_content) for doc in split_docs])}")

切分后的字符数（可以用来大致评估 token 数）：5408


注：如何对文档进行分割，其实是数据处理中最核心的一步，其往往决定了检索系统的下限。但是，如何选择分割方式，往往具有很强的业务相关性——针对不同的业务、不同的源数据，往往需要设定个性化的文档分割方式。因此，在本章，我们仅简单根据 chunk_size 对文档进行分割。对于有兴趣进一步探索的读者，欢迎阅读我们第三部分的项目示例来参考已有的项目是如何进行文档分割的。

# 批量数据预处理

In [1]:
import os
from dotenv import load_dotenv, find_dotenv 
# pip install python-dotenv

_ = load_dotenv(find_dotenv())


# 获取folder_path下所有文件路径，储存在file_paths里
file_paths = []
folder_path = '../../data_base/knowledge_db'
for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        file_paths.append(file_path)
print(file_paths[:3])

['../../data_base/knowledge_db\\pumkin_book\\pumpkin_book.pdf']


In [2]:
from langchain.document_loaders.pdf import PyMuPDFLoader
# from langchain.document_loaders.markdown import UnstructuredMarkdownLoader

# 遍历文件路径并把实例化的loader存放在loaders里
loaders = []

for file_path in file_paths:
    file_type = file_path.split('.')[-1]
    if file_type == 'pdf':
        loaders.append(PyMuPDFLoader(file_path))
    else:
        print(f"Unsupported file type: {file_type} for file {file_path}")

In [ ]:
# # 下载文件并存储到text
# # 未作数据清洗
# texts = []

# for loader in loaders:
#     texts.extend(loader.load())

# texts[2]

In [3]:
# 下载文件并存储到text
# 作数据清洗
from langchain.document_loaders.pdf import PyMuPDFLoader
import re
# 下载文件并存储到text
texts = []
for loader in loaders:
    texts.extend(loader.load())   

for text in texts:
    text.page_content = re.sub(r'\s*\n\s*', ' ', text.page_content)
    text.page_content = re.sub(r'[\s•]', '', text.page_content)

In [4]:
texts[100].page_content

'→_→欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》←_←当某一个类别j的基分类器的结果之和，大于所有结果之和的12，则选择该类别j为最终结果。8.4.5式(8.25)的解释H(x)=cargmaxj∑Ti=1hji(x)相比于其他类别，该类别j的基分类器的结果之和最大，则选择类别j为最终结果。8.4.6式(8.26)的解释H(x)=cargmaxj∑Ti=1wihji(x)相比于其他类别，该类别j的基分类器的结果之和最大，则选择类别j为最终结果，与式(8.25)不同的是，该式在基分类器前面乘上一个权重系数，该系数大于等于0，且T个权重之和为1。8.4.7元学习器(meta-learner)的解释书中第183页最后一行提到了元学习器(meta-learner)，简单解释一下，因为理解meta的含义有时对于理解论文中的核心思想很有帮助。元(meta)，非常抽象，例如此处的含义，即次级学习器，或者说基于学习器结果的学习器；另外还有元语言，就是描述计算机语言的语言，还有元数学，研究数学的数学等等；另外，论文中经常出现的还有meta-strategy，即元策略或元方法，比如说你的研究问题是多分类问题，那么你提出了一种方法，例如对输入特征进行变换（或对输出类别做某种变换），然后再基于普通的多分类方法进行预测，这时你的方法可以看成是一种通用的框架，它虽然针对多分类问题开发，但它需要某个具体多分类方法配合才能实现，那么这样的方法是一种更高层级的方法，可以称为是一种meta-strategy。8.4.8Stacking算法的解释该算法其实非常简单，对于数据集，试想你现在有了个基分类器预测结果，也就是说数据集中的每个样本均有个预测结果，那么怎么结合这个预测结果呢？本节名为“结合策略”，告诉你各种结合方法，但其实最简单的方法就是基于这个预测结果再进行一次学习，即针对每个样本，将这个预测结果作为输入特征，类别仍为原来的类别，既然无法抉择如何将这些结果进行结合，那么就“学习”一下吧。“西瓜书”图8.9伪代码第9行中将第个样本进行变换，特征为个基学习器的输出，类别标记仍为原来的，将所有训练集中的样本进行转换得到新的数据集后，再基于进行一次学习即可，也就是Stacking算法。至于说“西瓜书”图8.9中伪代码第1行到第3行使用的数据集与第5行到第10行使用的数据集之间的关系，在“西瓜

载入后的变量类型为`langchain_core.documents.base.Document`, 文档变量类型同样包含两个属性
- `page_content` 包含该文档的内容。
- `meta_data` 为文档相关的描述性数据。

In [5]:
text = texts[77]
print(f"每一个元素的类型：{type(text)}.", 
    # f"该文档的描述性数据：{text.metadata}", 
    f"查看该文档的内容:\n{text.page_content[0:]}", 
    sep="\n------\n")

每一个元素的类型：<class 'langchain_core.documents.base.Document'>.
------
查看该文档的内容:
→_→欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》←_←7.3.3贝叶斯估计[1]贝叶斯学派视角下的一类点估计法称为贝叶斯估计，常用的贝叶斯估计有最大后验估计（MaximumAPosterioriEstimation，简称MAP）、后验中位数估计和后验期望值估计这3种参数估计方法，下面给出这3种方法的具体定义。设总体的概率质量函数（若总体的分布为连续型时则改为概率密度函数，此处以离散型为例）为P(x|θ)，从该总体中抽取出的n个独立同分布的样本构成样本集D={x1,x2,···,xn}，则根据贝叶斯式可得，在给定样本集D的条件下，θ的条件概率为P(θ|D)=P(D|θ)P(θ)P(D)=P(D|θ)P(θ)PθP(D|θ)P(θ)其中P(D|θ)为似然函数，由于样本集D中的样本是独立同分布的，所以似然函数可以进一步展开，有P(θ|D)=P(D|θ)P(θ)PθP(D|θ)P(θ)=Qni=1P(xi|θ)P(θ)PθQni=1P(xi|θ)P(θ)根据贝叶斯学派的观点，此条件概率代表了我们在已知样本集D后对θ产生的新的认识，它综合了我们对θ主观预设的先验概率P(θ)和样本集D带来的信息，通常称其为θ的后验概率。贝叶斯学派认为，在得到P(θ|D)以后，对参数θ的任何统计推断，都只能基于P(θ|D)。至于具体如何去使用它，可以结合某种准则一起去进行，统计学家也有一定的自由度。对于点估计来说，求使得P(θ|D)达到最大值的ˆθMAP作为θ的估计称为最大后验估计，求P(θ|D)的中位数ˆθMedian作为θ的估计称为后验中位数估计，求P(θ|D)的期望值（均值）ˆθMean作为θ的估计称为后验期望值估计。7.3.4Categorical分布Categorical分布又称为广义伯努利分布，是将伯努利分布中的随机变量可取值个数由两个泛化为多个得到的分布。具体地，设离散型随机变量X共有k种可能的取值{x1,x2,···,xk}，且X取到每个值的概率分别为P(X=x1)=θ1,P(X=x2)=θ2,···,P(X=xk)=θk，则称随机变量X服从参数为θ1,θ2,···,θk的Categorical分布，其概率质量函数为P

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 切分文档
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=50)

split_docs = text_splitter.split_documents(texts)
print(f"切分后的文件数量：{len(split_docs)}")
print(f"切分后的字符数（可以用来大致评估 token 数）：{sum([len(doc.page_content) for doc in split_docs])}")

切分后的文件数量：595
切分后的字符数（可以用来大致评估 token 数）：252203
